In [1]:
from __future__ import division
import os
import pandas as pd
import numpy as np
from itertools import combinations
import operator

import itertools


In [2]:


def checkConfidence(body,frequencyset,conf,suppdict):
    confidence=suppdict.get(frequencyset)/(suppdict.get(body))
    if confidence*100 >= conf:
        return True
    else:
        return False

def checkMinSupForL1(geneCountDict,df, minsup):
    suppCountForL1 = {}
    minSupList = []
    for value in geneCountDict.keys():
      #  print(value)
        objMin = float(geneCountDict[value]/float(len(df)))
        if(objMin >= minsup):
            minSupList.append(value)
            suppCountForL1.update({value:objMin})
    return minSupList,suppCountForL1

def generateRulesForItemSets(freqItemSetListRules,supportCountDictionary,minConf):
    returnHeadResult =[]; returnBodyResult =[]
    for itemsList in freqItemSetListRules:
        headlist=[]
        bodylist=[]
        for i in range(len(itemsList)):
            copyIndividualSet = itemsList[:]
            headOfRule=itemsList[i:i+1]
            headOfRule=",".join(headOfRule)
            del copyIndividualSet[i]
            bodyOfRule=copyIndividualSet
            bodyOfRule=",".join(bodyOfRule)
            frequentStringSet = ",".join(itemsList)
            confidenceCheckForBody =  checkConfidence(bodyOfRule,frequentStringSet,minConf,supportCountDictionary)
            if(confidenceCheckForBody):
                if headOfRule not in headlist:
                    headlist.append(headOfRule)
                    bodylist.append(bodyOfRule)
            confidenceCheckForBody = checkConfidence(headOfRule,frequentStringSet,minConf,supportCountDictionary)
            if (confidenceCheckForBody):
                if headOfRule not in bodylist: 
                    headlist.append(bodyOfRule)
                    bodylist.append(headOfRule)
            if len(itemsList)>2:
                for m in range(i+1, len(itemsList)):
                    temp=[]   
                    copyIndividualSet = itemsList[:]
                    temp.append(itemsList[i])
                    itemsList.append(itemsList[m])
                    copyIndividualSet.remove(itemsList[i])
                    copyIndividualSet.remove(itemsList[m])
                    temp=",".join(temp)
                    copyIndividualSet=",".join(copyIndividualSet)
                    if checkConfidence(copyIndividualSet,frequentStringSet,minConf,supportCountDictionary):
                        if temp not in headlist:
                            headlist.append(temp)
                            bodylist.append(copyIndividualSet)
                    if checkConfidence(temp,frequentStringSet,minconf,supportCountDictionary):
                        if temp not in bodylist:
                            bodylist.append(temp)
                            headlist.append(copyIndividualSet)
                 
        returnHeadResult = returnHeadResult + headlist
        returnBodyResult = returnBodyResult + bodylist
    return returnHeadResult, returnBodyResult





def calculateSupport(twopair,geneDataList):
    support = 0
    for data in geneDataList:
        #print(twopair)
        #print(data)
        if set(twopair).issubset(data):
            support = support + 1
   # print("support",support)
    return support

def generateSetsForK(freqItemSetList,k,minsuppo,geneDataList):
    
    
    isElementMatch = False
    returnList = []
    p = 0
    suppCountDict = {}
    for i in freqItemSetList[p:]:
        for j in freqItemSetList[p+1:]:
            if(k == 2):
                twopair = []
                twopair.append(i);
                twopair.append(j);
                    #print(type(twopair))
                supportCountForPair = float(calculateSupport(twopair,geneDataList)/100)
                if(supportCountForPair>=minsuppo):
                    returnList.append(twopair)
                    suppCountDict.update({",".join(twopair):supportCountForPair})
            else:
                
                for index in range(k-2):
                    if(i[index] != j[index]):
                        isElementMatch = True
                        break
                if (isElementMatch):
                    isElementMatch = False
                    break
                else:
                    itemSet = []
                    itemSet = itemSet + i
                    itemSet.append(j[len(j) - 1])
                    supportCount = float(calculateSupport(itemSet, geneDataList)/100)
                    if (supportCount >= minsuppo):
                        returnList.append(itemSet)
                        suppCountDict.update({",".join(itemSet):supportCount})
        p = p+1
    return returnList, suppCountDict 
                
    
    
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".txt"):
        currentFile = os.getcwd()+'/'+filename
        print("------------------------------------------------------------------------------------")
        print("NOW WORKING ON FILE "+filename)
        print("------------------------------------------------------------------------------------")
        df = pd.read_csv(currentFile, sep='\t', header = None)

        for i in range(len(df)):
            for j in range(len(df[i])):
                df[i][j] = "G"+str(i+1)+"_"+df[i][j]
             
        dict = {}
        for i in range(df.shape[0]):
            for j in range(df.shape[1]-1):
                key = df[i][j]
                if key in dict.keys():
                    dict[key] = dict[key] + 1
                else:
                    dict.update( {key : 1} )
                    
        for key in df[100]:
            if key in dict.keys():
                dict[key] = dict[key] + 1
            else:
                dict.update( {key : 1} )
                
        
        all_word_dict = {}
        for i in range(df.shape[0]):
            row = []
            for j in range(df.shape[1]-1):
                row.append(df[j][i]);
            all_word_dict.update( {i : row} )

        geneDataList = []
        for i in all_word_dict.keys():
            rowSet = set()
            for data in all_word_dict[i]:
                rowSet.add(data)
            geneDataList.append(rowSet)
            
            #minsupp put here in value/100
        minsupp=0.7
        supportCountDictionary = {}
        initialFrquentItems,suppCountForL1 = checkMinSupForL1(dict,df,minsupp)
        supportCountDictionary.update(suppCountForL1)
        freqItemSetList = [initialFrquentItems]
        freqItemSetListRules = []
        k=2
        sizeChecker = 1
        while(sizeChecker>0):
            newItemSetList,suppCount = generateSetsForK(freqItemSetList[k-2],k,minsupp,geneDataList)
            supportCountDictionary.update(suppCount)
            sizeChecker = len(newItemSetList)
            if sizeChecker > 0:
                freqItemSetList.append(newItemSetList)
                freqItemSetListRules = freqItemSetListRules + newItemSetList
            k += 1
#minconfidence here in %
        minconfidence = 70
        rules_head, rules_body= generateRulesForItemSets(freqItemSetListRules, supportCountDictionary, minconfidence)
            
        print("FINISHED WORKING ON FILE "+filename+"\n")
    else:
        continue

------------------------------------------------------------------------------------
NOW WORKING ON FILE associationruletestdata.txt
------------------------------------------------------------------------------------
FINISHED WORKING ON FILE associationruletestdata.txt



In [3]:
70
def template1(par1,par2,par3,head,body):
    result = []
    total_count = 0
    if par1 == "RULE":
        if par2 == "ANY":
            for i in par3:
                for j in range(len(head)):
                    if i in head[j]:
                        result.append(body[j]+'->'+head[j])
                        total_count = total_count + 1
                    elif i in body[j]:
                        result.append(body[j]+'->'+head[j])
                        total_count = total_count + 1
        elif par2 == "NONE":
            for i in par3:
                for j in range(len(head)):
                    if i in head[j] or i in body[j]:
                        break;
                    else: 
                        result.append(body[j]+'->'+head[j])
                        total_count = total_count + 1
        elif par2 == 1:
            for i in par3:
                for j in range(len(head)):
                    if i in head[j]:
                        result.append(body[j]+'->'+head[j])
                        total_count = total_count + 1
                    elif i in body[j]:
                        result.append(body[j]+'->'+head[j])
                        total_count = total_count + 1
    
    elif par1 == "HEAD":
        if par2 == "ANY":
            for i in par3:
                for j in range(len(head)):
                    if i in head[j]:
                        result.append(body[j]+'->'+head[j])
                        total_count = total_count + 1
                    
        elif par2 == "NONE":
            for i in par3:
                for j in range(len(head)):
                    if i in head[j]:
                        break;
                    else: 
                        result.append(body[j]+'->'+head[j])
                        total_count = total_count + 1
        elif par2 == 1:
            for i in par3:
                for j in range(len(head)):
                    if i in head[j]:
                        result.append(body[j]+'->'+head[j])
                        total_count = total_count + 1

    elif par1 == "BODY":
        if par2 == "ANY":
            for i in par3:
                for j in range(len(body)):
                    if i in body[j]:
                        result.append(body[j]+'->'+head[j])
                        total_count = total_count + 1
                    
        elif par2 == "NONE":
            for i in par3:
                for j in range(len(body)):
                    if i in body[j]:
                        break;
                    else: 
                        result.append(body[j]+'->'+head[j])
                        total_count = total_count + 1
        elif par2 == 1:
            for i in par3:
                for j in range(len(body)):
                    if i in body[j]:
                        result.append(body[j]+'->'+head[j])
                        total_count = total_count + 1
                        
    return result, total_count



def template2(par1,par2,head,body):
    result = []
    total_count = 0
    if par1 == "RULE":
        for i in range(len(body)):
            if(len(head[i])+len(body[i]) >= par2):
                result.append(body[i]+'->'+head[i])
                total_count = total_count + 1

    elif par1 == "BODY":
        for i in range(len(body)):
            if(len(body[i]) >= par2):
                result.append(body[i]+'->'+head[i])
                total_count = total_count + 1

    elif par1 == "HEAD":
        for i in range(len(head)):
            if(len(head[i]) >= par2):
                result.append(body[i]+'->'+head[i])
                total_count = total_count + 1
    return result, total_count

def template3(head,body,cond, par1, par2, par3, par4, par5='0', par6='0'):
    result = set()
    if cond =='1or1':
        ans1 = template1(par1, par2, par3,head,body)
        ans2 = template1(par4, par5, par6,head,body)
        result = set(ans1[0]).union(set(ans2[0]))
    
    elif cond =='1and1':
        ans1 = template1(par1, par2, par3,head,body)
        ans2 = template1(par4, par5, par6,head,body)
        result = set(ans1[0]).intersection(set(ans2[0]))
    
    elif cond =='1or2':
        ans1 = template1(par1, par2, par3,head,body)
        ans2 = template2(par4, par5, head,body)
        result = set(ans1[0]).union(set(ans2[0]))
    
    elif cond =='1and2':
        ans1 = template1(par1, par2, par3,head,body)
        ans2 = template2(par4, par5, head,body)
        result = set(ans1[0]).union(set(ans2[0]))
    
    elif cond =='2or2':
        ans1 = template2(par1, par2, head,body)
        ans2 = template2(par3, par4, head,body)
        result = set(ans1[0]).union(set(ans2[0]))
    
    elif cond =='2and2':
        ans1 = template2(par1, par2, head,body)
        ans2 = template2(par3, par4, head,body)
        result = set(ans1[0]).union(set(ans2[0]))
    return result, len(result)

In [4]:
print("-----------------------------------------------------------")
print("template1")
print("-----------------------------------------------------------")

print("-----------------------------------------------------------")
print("1")
print(template1("RULE", "ANY", ['G59_UP'], rules_head, rules_body))
print("-----------------------------------------------------------")
print("2")
print(template1("RULE", "NONE", ['G59_UP'], rules_head, rules_body))
print("-----------------------------------------------------------")
print("3")
print(template1("RULE", 1, ['G59_UP', 'G10_Down'], rules_head, rules_body))
print("-----------------------------------------------------------")
print("4")
print(template1("HEAD", "ANY", ['G59_UP'], rules_head, rules_body))
print("-----------------------------------------------------------")
print("5")
print(template1("HEAD", "NONE", ['G59_UP'], rules_head, rules_body))
print("-----------------------------------------------------------")
print("6")
print(template1("HEAD", 1, ['G59_UP', 'G10_Down'], rules_head, rules_body))
print("-----------------------------------------------------------")
print("7")
print(template1("BODY", "ANY", ['G59_UP'], rules_head, rules_body))
print("-----------------------------------------------------------")
print("8")
print(template1("BODY", "NONE", ['G59_UP'], rules_head, rules_body))
print("-----------------------------------------------------------")
print("9")
print(template1("BODY", 1, ['G59_UP', 'G10_Down'], rules_head, rules_body))


print("-----------------------------------------------------------")
print("template2")
print("-----------------------------------------------------------")
print("-----------------------------------------------------------")
print("1")
print(template2("RULE", 3, rules_head, rules_body))
print("-----------------------------------------------------------")
print("2")
print(template2("HEAD", 2, rules_head, rules_body))
print("-----------------------------------------------------------")
print("3")
print(template2("BODY", 1, rules_head, rules_body))

print("-----------------------------------------------------------")
print("template3")
print("-----------------------------------------------------------")
print("-----------------------------------------------------------")
print("1")
print("-----------------------------------------------------------")
print(template3(rules_head, rules_body,"1or1", "HEAD", "ANY",['G10_Down'], "BODY", 1, ['G59_UP']))
print("-----------------------------------------------------------")
print("2")
print("-----------------------------------------------------------")
print(template3(rules_head, rules_body,"1and1", "HEAD", "ANY",['G10_Down'], "BODY", 1, ['G59_UP']))
print("-----------------------------------------------------------")
print("3")
print("-----------------------------------------------------------")
print(template3(rules_head, rules_body,"1or2", "HEAD", "ANY",['G10_Down'], "BODY", 2))
print("-----------------------------------------------------------")
print("4")
print("-----------------------------------------------------------")
print(template3(rules_head, rules_body,"1and2", "HEAD", "ANY",['G10_Down'], "BODY", 2))
print("-----------------------------------------------------------")
print("5")
print("-----------------------------------------------------------")
print(template3(rules_head, rules_body,"2or2", "HEAD", 1, "BODY", 2))
print("-----------------------------------------------------------")
print("6")
print("-----------------------------------------------------------")
print(template3(rules_head, rules_body,"2and2", "HEAD", 1, "BODY", 2))





-----------------------------------------------------------
template1
-----------------------------------------------------------
-----------------------------------------------------------
1
([], 0)
-----------------------------------------------------------
2
([], 0)
-----------------------------------------------------------
3
([], 0)
-----------------------------------------------------------
4
([], 0)
-----------------------------------------------------------
5
([], 0)
-----------------------------------------------------------
6
([], 0)
-----------------------------------------------------------
7
([], 0)
-----------------------------------------------------------
8
([], 0)
-----------------------------------------------------------
9
([], 0)
-----------------------------------------------------------
template2
-----------------------------------------------------------
-----------------------------------------------------------
1
([], 0)
----------------------------------------